## Introduction to Databases with Python
----------------------------------------------------------------------------------------

![sqlitepython](http://www.sqlitetutorial.net/wp-content/uploads/2016/01/SQLite-Python.jpg)

# WTF is a database, dude?
----------------------------------------------------------------------------------------
* A collection of data, can scale better than simple(flat) file.
* Usually designed to model real world(OOP someone?).
* Offers some kind of query language for CRUD operations.
* Most popular are the relational databases.

# Now WTF is a relational database?
-----------------------------------------------------------------------------------------

> This(relational) model organizes data into one or more tables (or "relations") of columns and rows, with a unique key identifying each row. Rows are also called records or tuples.[2] Columns are also called attributes. Generally, each table/relation represents one "entity type" (such as customer or product). The rows represent instances of that type of entity (such as "Lee" or "chair") and the columns representing values attributed to that instance (such as address or price).
                                                                                    Wikipedia
                                                                                    
* Relationships between entities can solve real world problems(e.g. John(**entity-customer**) buys 1 Big Mac(**entity-product**).)
* The query language is **SQL**- many different flavors but the underlying concepts remain the same.

# Why it is useful?
-----------------------------------------------------------------------------------------
![](https://www.lee.k12.nc.us/cms/lib/NC01001912/Centricity/Domain/2133/suggestions.jpg)

## Just because it is
------------------------------------------------------------------------------------------

* Separation of program logic(code) and data.
* Same data can be reused in many different applications.
* More sophisticated and flexible than flat file. What do you think if 20+ people are opening and saving the same file at the same time? What if somebody forgets to close it? 

Remember:
* Files corrupt easily and generally are not designed for concurrent multi-user read-write. 
* Also when you open them you load the whole thing in your RAM memory. Not the case with databases.
* How do you query a file? Errr....let's just use SQLite.

# What is SQLite 3?
----------------------------------------------------------------------------------------------

* Relational database engine. Comes with your Python installation.
* Your whole database is kept in a single file.
* Suitable for small to medium scale projects.

![signmeup](http://greatbridgewrestlingclub.com/wp-content/uploads/2014/08/sign-me-up.jpg)

# Using SQLite with Python
------------------------------------------------------------------------------

```python
import sqlite3

conn = sqlite3.connect('progress.db')
c = conn.cursor()
```

Now go check the folder your program is in. There is another file there. Takeout: if we connect to a database that does not yet exist, SQLite creates it.

Also you can create a in-memory database using the following connection string:
```
c1 = sqlite3.connect("file::memory:?cache=shared")
```
By default SQLite does not offer any password protection so whoever has the db file has your data. Not perfect, but many solutions. You can research the official encryption extension [here](http://www.hwaci.com/sw/sqlite/see.html).

# I want to create and update a table
----------------------------------------------------------------------------------------------------------------

All the transactions are executed using the cursor. Let's see the official example from the docs:
```python
# Create table
c.execute('''CREATE TABLE IF NOT EXISTS stocks
             (date text, trans text, symbol text, qty real, price real)''')

# Insert a row of data
c.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()
```
See how after transactions are executed, they have to be commited to take effect. Also like we did with files we have to close the connection when we are done.

# Onto the cool stuff now
----------------------------------------------------------------------------------------
We can execute database queries in functions and with variables from our program. 
```python
def insert_row(date,trans,symbol,qty,price):
    c.execute("INSERT INTO stocks (date,trans,symbol,qty,price) VALUES (?,?,?,?,?)",(date,trans,symbol,qty,price))
    c.commit()
    
insert_row('2011-01-05','SELL','APPL',10000,11.12)
```
In order to make the execute more Python 3 looking we can do:
```python
c.execute("INSERT INTO stocks (date,trans,symbol,qty,price) VALUES ('{}','{}','{}',{},{})".format(date,trans,symbol,qty,price))
```
Note that setting database entries using variables this way can break the whole database. 
We can select entres from the database:
```python
c.execute('SELECT * from stocks')
for row in c.fetchall()
    print(row)
```
Try to execute the following query by yourself: SELECT * FROM stocks WHERE trans='BUY'
Then try SELECTing just one column from the table.

In [1]:
```python
c.execute('SELECT * FROM stocks where trans=?',('BUY',))

SyntaxError: invalid syntax (<ipython-input-1-4ec236970c92>, line 1)

# Context managers a.k.a. with statement and SQLite
-----------------------------------------------------------------------

```python
def insert_row(date,trans,symbol,qty,price):
    with conn:
        c.execute("INSERT INTO stocks (date,trans,symbol,qty,price) VALUES (?,?,?,?,?)",(date,trans,symbol,qty,price))
```

# UPDATE and DELETE - point of no return
-----------------------------------------------------------------------------

```python
c.execute('UPDATE stocks SET price=1 WHERE price=11.12')
c.commit()
```

```python
c.execute('DELETE FROM stocks WHERE price=1')
c.commit()
```

# DUMPing database to a file
-----------------------------------------------------------------------------
```python
with open('dump.sql','w') as f:
    for line in conn.iterdump():
        f.write("{}\n".format(line))
```